# Partie 1

## Modules

In [35]:
import openpyxl
from classes import Ressource,RessourceUnavailability,Task,TaskUnavailability
from read_input import read_ressources,read_ressources_unavailabilities,read_tasks,read_tasks_unavailabilities

import gurobipy as gp
from gurobipy import GRB

## Lecture des données

In [36]:
ressources = read_ressources(1)
ressources_unavailabilities = read_ressources_unavailabilities(1)
tasks = read_tasks(1)
tasks_unavailabilities = read_tasks_unavailabilities(1)

NameError: name 'openpyxl' is not defined

## Structure de données

In [4]:
R = [j for j in ressources]
T = [i for i in tasks]

# creer locations
# creer dist

# https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html


## Modèle

In [7]:
### Create Model
m = gp.Model("trs0")

### Decision variables
# Task-ressource assignment
x = m.addVars(R, T, vtype=GRB.BINARY, name="x")
# Edge-route assignment to ressource
y = m.addVars(L, L, R, vtype=GRB.BINARY, name="y")

# Technician cannot leave or return to a depot that is not its base
for r in R:
  for d in D:
    if k.depot != d:
      for i in L:
        y[i,d,r].ub = 0
        y[d,i,r].ub = 0

NameError: name 'L' is not defined